## Загрузка данных

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn

from math import ceil
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler 

In [ ]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/6_class.csv', index_col='Unnamed: 0')

## Предобработка и анализ данных

In [ ]:
df.head()

In [ ]:
def num(column):
    print(column,  '\\n')
    print(df[column].describe())

    figure, axis = plt.subplots(1, 2, figsize=(13,5))

    plt.style.use('bmh')

    axis[0].hist(df[column])
    axis[0].set_title('График распределения значений')

    axis[1].boxplot(df[column])
    axis[1].set_title('График размаха значений')

    figure.suptitle(column)
    plt.show()

In [ ]:
numeric = ['Temperature (K)', 'Luminosity(L/Lo)', 'Radius(R/Ro)', 'Absolute magnitude(Mv)']

In [ ]:
for col in numeric:
    num(col)

### Выводы по количестенным признакам

1. Основная масса представленных температур звезд находится в диапозоне до 6и тысяч. Низкая представленность других объектов.
2. По светимости ситуация похожая, больше половины объектов практически не имеют светимости.
3. Почти 90% объектов имеют радиус меньше 250, представленнось других объектов минимальна.
4. Масса звезд распределенна округ двух показателей: около -7 и около 12.

In [ ]:
def cat(column):
    print(column, '\\n')
    print(df[column].value_counts())

    names = list(df[column].value_counts().index)
    values = list(df[column].value_counts().values)

    figure, axis = plt.subplots(figsize=(5, 5))

    axis.barh(names, values)

    plt.style.use('bmh')

    figure.suptitle(column)
    plt.show()

In [ ]:
categorical = ['Star type', 'Star color']

In [ ]:
df['Star color'] = df['Star color'].str.lower().str.strip()

In [ ]:
df['Star color'].unique()

In [ ]:
df['Star color'] = df['Star color'].replace({'blue white': 'blue',
                                             'yellowish white': 'yellow',
                                             'pale yellow orange': 'yellow',
                                             'blue-white': 'blue',
                                             'whitish': 'white',
                                             'yellow-white': 'yellow',
                                             'white-yellow': 'yellow',
                                             'yellowish': 'yellow',
                                             'orange-red': 'red'})

In [ ]:
for col in categorical:
    cat(col)

### Выводы по категориальным признакам

1. Распределение объектов по типу является равномерным.
2. 90% объектов являются голубыми или красными планетами.

In [ ]:
y = df["Temperature (K)"]
X = df.drop("Temperature (K)", axis=1)

In [ ]:
X['Star color'] = OrdinalEncoder().fit_transform(X[['Star color']])

In [ ]:
scaler = StandardScaler()
columns_X = X.columns
X = scaler.fit_transform(X)
X = pd.DataFrame(X, columns=columns_X)

del scaler
del columns_X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, shuffle=True)

In [ ]:
X_train_tensor = torch.FloatTensor(X_train.values) 
X_test_tensor = torch.FloatTensor(X_test.values)
y_train_tensor = torch.FloatTensor(y_train.values)
y_test_tensor = torch.FloatTensor(y_test.values)

### Выводы по данным

Колличество данных является небольшим и большенство из их признаков сконцентрированны у одних показателей, что может негативно сказаться на качестве предсказаний у объектов с низкой представленностью.

## Построение базовой нейронной сети

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

In [ ]:
num_epochs = 1000
learning_rate = 0.2

In [ ]:
def init_weights(layer):
    if type(layer) == nn.Linear: 
        nn.init.normal_(layer.weight, mean=0.0, std=1.14)
        nn.init.normal_(layer.bias, mean=-0.5, std=1.0)

In [ ]:
results_loss_1 = [] 
results_time_1 = [] 
results_loss_2 = [] 

n_in_neurons = 5
n_hidden_neurons_1 = 10
n_hidden_neurons_2 = 10
n_out_neurons = 1 

net = nn.Sequential(
    nn.Linear(n_in_neurons, n_hidden_neurons_1),
    nn.ELU(),
    nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2),
    nn.ELU(),
    nn.Linear(n_hidden_neurons_2, n_out_neurons),
    nn.ReLU()
)

net.apply(init_weights)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
loss = nn.MSELoss()

In [ ]:
for epoch in range(num_epochs):
    net.train()
    optimizer.zero_grad()
    preds = net.forward(X_train_tensor).flatten()
        
    loss_value = loss(preds, y_train_tensor)
    loss_value.backward()
    optimizer.step()

    with torch.no_grad():   
            net.eval()
            test_preds = net.forward(X_test_tensor).flatten()
            loss_value_test = loss(test_preds, y_test_tensor) 
            print('epoch {}, RMSE train {:.4f}, RMSE test {:.4f}'.format(epoch, torch.Tensor.sqrt_(loss_value),\
                                                                            torch.Tensor.sqrt_(loss_value_test)))                
            
            results_loss_1.append(loss_value.tolist())
            results_time_1.append(epoch)
            results_loss_2.append(loss_value_test.tolist())
                
results_loss_1 = pd.Series(results_loss_1)
results_time_1 = pd.Series(results_time_1)
results_loss_2 = pd.Series(results_loss_2)

best_idx = results_loss_2.idxmin()

print('')
print('Лучшее значение RMSE test: ', results_loss_2[best_idx])
print('RMSE train: ', results_loss_1[best_idx])
print('Epoch: ', results_time_1[best_idx])

In [ ]:
def build_graphic(target, preds):
    figure, axis = plt.subplots(1, 1, figsize=(32,8))
    
    plt.style.use('bmh')

    axis.bar(x = target.index, height = preds.int(), width = 3, alpha=0.5, label = 'Прогноз', color='blue')
    axis.bar(x = target.index, height = target.values, width = 1, label = 'Факт', color='orange')

    plt.xlabel("Номер звезды в таблице данных")
    plt.ylabel("Температура звезды")

    figure.suptitle('График')
    plt.legend()
    plt.show()

In [ ]:
build_graphic(y_test, test_preds)

### Выводы по бэйзлайну,

Базоная нейронная сеть быстро обучается и показывает хорошие результаты на тестовой выборке.
Скорость обучения достигается за счет небольшого количества данных и небольшого количества признаков.
Можно заметить, что объекты с низкой представленностью в данных показывают худшие результаты, а именно объекты с высокой температурой.

## Улучшение нейронной сети

In [ ]:
X_train_n, X_valid_n, y_train_n, y_valid_n = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, shuffle=True)

print(X_train_n.shape, y_train_n.shape, X_valid_n.shape, y_valid_n.shape)
X_train_n_tensor = torch.FloatTensor(X_train_n.values) 
X_valid_n_tensor = torch.FloatTensor(X_valid_n.values)
y_train_n_tensor = torch.FloatTensor(y_train_n.values)
y_valid_n_tensor = torch.FloatTensor(y_valid_n.values)

In [ ]:
results_loss_1 = [] 
results_time_1 = [] 
results_loss_2 = []
do1 = []
do2 = []
preds_2 = []

p1 = [.1, .2, .5, .8]
p2 = p1

num_epochs = 5_000

p1 = [.2, .5, .8]
p2 = p1

num_epochs = 10_000

for v1 in p1:
        for v2 in p2:
                net2 = nn.Sequential(
                nn.Linear(n_in_neurons, n_hidden_neurons_1),
                nn.ELU(),
                nn.Dropout(p=v1),
                nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2),
                nn.ELU(),
                nn.Dropout(p=v2),
                nn.Linear(n_hidden_neurons_2, n_out_neurons),
                nn.ReLU()
                )

                net2.apply(init_weights)
                optimizer = torch.optim.Adam(net2.parameters(), lr=learning_rate)
                loss = nn.MSELoss()
                for epoch in range(num_epochs):
                        net2.train()
                        optimizer.zero_grad()
                        preds = net2.forward(X_train_n_tensor).flatten()
                        
                        loss_value = loss(preds, y_train_n_tensor)
                        loss_value.backward()
                        optimizer.step()

                        with torch.no_grad():   
                                net2.eval()
                                valid_preds = net2.forward(X_valid_n_tensor).flatten()
                                loss_value_valid = loss(valid_preds, y_valid_n_tensor) 
                                print('Do1 {}, Do2 {}, epoch {}, RMSE train {:.4f}, RMSE valid {:.4f}'.format(v1, v2, epoch, torch.Tensor.sqrt_(loss_value),\
                                        torch.Tensor.sqrt_(loss_value_valid)))                
                        
                                
                                results_loss_1.append(loss_value.tolist())
                                results_time_1.append(epoch)
                                results_loss_2.append(loss_value_valid.tolist())
                                do1.append(v1)
                                do2.append(v2)
                                preds_2.append(valid_preds)

results_loss_1 = pd.Series(results_loss_1)
results_time_1 = pd.Series(results_time_1)
results_loss_2 = pd.Series(results_loss_2)
do1 = pd.Series(do1)
do2 = pd.Series(do2)

best_idx = results_loss_2.idxmin()
        
print('')
print('Лучшее значение RMSE valid: ', results_loss_2[best_idx])
print('RMSE train: ', results_loss_1[best_idx])
print('Dropout 1: ', do1[best_idx])
print('Dropout 2: ', do2[best_idx])
print('Epoch: ', results_time_1[best_idx])

In [ ]:
build_graphic(y_valid_n, preds_2[best_idx])

### Выводы по модели с подбором Дропаута

Из-за применения дропаута, модель использует в 3 раза больше эпох для обучения, но показывает немногого улучшенные показатели ошибки на валидационной выборке при дропауте в 10%.
Проблема с низкой представленностью объектов также влияет на качество предсказаний.

In [ ]:
results_loss_3 = []
results_time_3 = []
results_size_3 = []
result_loss_value_3 = []
preds_3 = []

net3 = nn.Sequential(
    nn.Linear(n_in_neurons, n_hidden_neurons_1),
    nn.ELU(),
    nn.BatchNorm1d(n_hidden_neurons_1),    
    nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2),
    nn.ELU(),
    nn.BatchNorm1d(n_hidden_neurons_2),    
    nn.Linear(n_hidden_neurons_2, n_out_neurons),
    nn.ReLU()
)


bs = [5, 10, 25, 100]

net3.apply(init_weights)
optimizer = torch.optim.Adam(net3.parameters(), lr=learning_rate)
loss = nn.MSELoss()

for batch_size in bs:
    num_batches = ceil(len(X_train_n_tensor)/batch_size)

    for epoch in range(num_epochs):
        order = np.random.permutation(len(X_train_n_tensor))
        net3.train()
        optimizer.zero_grad()
        for batch_i in range(num_batches):
            start_index = batch_i * batch_size
            
            batch_indexes = order[start_index:start_index+batch_size]
            X_batch = X_train_n_tensor[batch_indexes]
            y_batch = y_train_n_tensor[batch_indexes]
        
            preds = net3.forward(X_batch) 
                    
            loss_value = loss(preds, y_batch)
            loss_value.backward()
            optimizer.step()

            with torch.no_grad():
                    
                        net3.eval()
                        test_preds = net3.forward(X_valid_n_tensor).flatten()
                        loss_value_valid = loss(test_preds, y_valid_n_tensor)

                        print('size {}, epoch {}, RMSE train {:.4f}, RMSE valid {:.4f}'.format(batch_size, epoch, \
                                torch.Tensor.sqrt_(loss_value), torch.Tensor.sqrt_(loss_value_valid)))
                        
                        results_loss_3.append(loss_value_valid.tolist())
                        result_loss_value_3.append(loss_value.tolist())
                        results_time_3.append(epoch)
                        results_size_3.append(batch_size)
                        preds_3.append(valid_preds)

results_loss_3 = pd.Series(results_loss_3)
result_loss_value_3 = pd.Series(result_loss_value_3)
results_time_3 = pd.Series(results_time_3)
results_size_3 = pd.Series(results_size_3)

best_idx = results_loss_3.idxmin()


print('')
print('Лучшее значение RMSE valid: ', results_loss_3[best_idx])
print('RMSE train: ', result_loss_value_3[best_idx])
print('Epoch: ', results_time_3[best_idx])
print('Batch size: ', results_size_3[best_idx])

In [ ]:
build_graphic(y_valid_n, preds_3[best_idx])

### Выводы по модели с подбором размера батч
    
Из-за использования батчей модель показывает худшие результаты на валидационной выборке, чем без батчей. Скорее всего это связано с распределением признаков. Они влияют и на другие архитектуры, но, возможно, на подход с батчами сильнее всего.

## Выводы

In [ ]:
final_epoches = 1815

final_net = nn.Sequential(
            nn.Linear(n_in_neurons, n_hidden_neurons_1),
            nn.ELU(),
            nn.Dropout(p=.1),
            nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2),
            nn.ELU(),
            nn.Dropout(p=.1),
            nn.Linear(n_hidden_neurons_2, n_out_neurons),
            nn.ReLU()
        )

final_net.apply(init_weights)
optimizer = torch.optim.Adam(final_net.parameters(), lr=learning_rate)
loss = nn.MSELoss()

In [ ]:
for epoch in range(final_epoches):
    final_net.train()
    optimizer.zero_grad()
    final_train_preds = final_net.forward(X_train_tensor).flatten()
    
    loss_value = loss(final_train_preds, y_train_tensor)
    loss_value.backward()
    optimizer.step()

In [ ]:
final_net.eval()
final_test_preds = final_net.forward(X_test_tensor).flatten()
loss_value_valid = loss(final_test_preds, y_test_tensor) 
print('Финальное значение RMSE : ', torch.Tensor.sqrt_(loss_value_valid))

Основной критерий влияющий на качество оценки модели это представленность объектов в данных. Объекты редко представленные в датасете показывают худшие результаты. Если и в будущем объектов, с которыми нужно работать, будет не много, то стоит усложнить модель и подобрать новые гиперпараметры для улучшения качества предсказаний.